<a href="https://colab.research.google.com/github/Prateeklk/llm-prompt-engineering/blob/main/lora_peft_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      

In [2]:
from datasets import load_dataset, DatasetDict, Dataset
# from transformers import AutoModelForSeq2SeqLM
# from transformers import AutoTokenizer
# from transformers import GenerationConfig

In [3]:
from transformers import(
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

In [4]:
!pip install peft

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00


In [27]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [7]:
dataset = load_dataset('shawhin/imdb-truncated')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [8]:
# trainabl data with label as 1
np.array(dataset['train']['label']).sum() / len(dataset['train']['label'])

0.5

## Model

In [9]:
# base model
model_checkpoint = 'distilbert-base-uncased'

# label map
id2label = {0:'Negative', 1: 'Positive'}
label2id = {'Negative':0, 'Positive':1}

'''The class automatically selects the correct model architecture based on
the specified checkpoint, saving you the hassle of manual configuration.'''

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels = 2,
                                                           id2label = id2label,
                                                           label2id = label2id)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# model architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [11]:
# data preprocessing
'''When you set add_prefix_space=True in AutoTokenizer.from_pretrained(), you're
 essentially instructing the tokenizer to add a space before every token it encounters.
This is particularly useful when dealing with models that expect input tokens to be separated by spaces, such as BERT-based models.'''
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space = True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
# add pad to the short token
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  model.resize_token_embedding(len(tokenizer))

In [13]:
def tokenize_function(examples):
  text = examples['text']

  tokenizer.truncation_side = 'left'
  tokenized_inputs = tokenizer(
      text,
      return_tensors = "np",
      truncation = True,
      max_length = 512

  )
  return tokenized_inputs

In [14]:
tokenzied_dataset = dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
tokenzied_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [16]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

### Evaluation

In [17]:
accuracy = evaluate.load("accuracy")

In [18]:
def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions , axis = 1)
  return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [19]:
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]


In [20]:
for text in text_list:
  inputs = tokenizer.encode(text, return_tensors = 'pt')
  logits = model(inputs).logits
  predictions = torch.argmax(logits)
  print(text + " - " + id2label[predictions.tolist()])

It was good. - Negative
Not a fan, don't recommed. - Negative
Better than the first one. - Negative
This is not worth watching even once. - Negative
This one is a pass. - Negative


## Train model

In [21]:
peft_config = LoraConfig(task_type = "SEQ_CLS",
                         r = 4,
                         lora_alpha = 32,
                         lora_dropout = 0.01,
                         target_modules =  ['q_lin']) #revisit it

In [22]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [23]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [24]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [25]:
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
trainer = Trainer(
    model=model,

    args=training_args,
    train_dataset=tokenzied_dataset["train"],
    eval_dataset=tokenzied_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.306680,{'accuracy': 0.896}
2,0.433600,0.475695,{'accuracy': 0.888}


Trainer is attempting to log a value of "{'accuracy': 0.896}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.888}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.306680,{'accuracy': 0.896}
2,0.433600,0.475695,{'accuracy': 0.888}
3,0.433600,0.614415,{'accuracy': 0.881}
4,0.180100,0.611103,{'accuracy': 0.888}
5,0.180100,0.839231,{'accuracy': 0.884}
6,0.056900,0.824803,{'accuracy': 0.892}
7,0.056900,0.924709,{'accuracy': 0.88}
8,0.018700,0.970023,{'accuracy': 0.887}
9,0.018700,1.009049,{'accuracy': 0.882}
10,0.005700,1.008255,{'accuracy': 0.881}


Trainer is attempting to log a value of "{'accuracy': 0.881}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.888}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.884}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.892}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.88}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This in

TrainOutput(global_step=2500, training_loss=0.1390057099342346, metrics={'train_runtime': 463.4019, 'train_samples_per_second': 21.58, 'train_steps_per_second': 5.395, 'total_flos': 1112883852759936.0, 'train_loss': 0.1390057099342346, 'epoch': 10.0})

In [32]:
import torch

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device (GPU in this case)
model.to(device)

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    # Tokenize the input text and move it to the device
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)

    # Perform the forward pass and get the logits
    logits = model(inputs).logits

    # Get the index of the max logit value (the predicted class)
    predictions = torch.max(logits, 1).indices

    # Print the text and the corresponding predicted label
    print(text + " - " + id2label[predictions.tolist()[0]])


Trained model predictions:
--------------------------
It was good. - Positive
Not a fan, don't recommed. - Negative
Better than the first one. - Positive
This is not worth watching even once. - Positive
This one is a pass. - Negative


In [33]:
list_text = ["the movie is decent","definitely watch it",
             "just waste of time"," not as good as i was expecting",
             "don't waste time just go a watch it!!"]

In [34]:

print("Trained model predictions:")
print("--------------------------")
for text in list_text:
    # Tokenize the input text and move it to the device
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)

    # Perform the forward pass and get the logits
    logits = model(inputs).logits

    # Get the index of the max logit value (the predicted class)
    predictions = torch.max(logits, 1).indices

    # Print the text and the corresponding predicted label
    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
the movie is decent - Negative
definitely watch it - Positive
just waste of time - Negative
 not as good as i was expecting - Negative
don't waste time just go a watch it!! - Positive
